# Building Effective Agents (with Pydantic AI)

Examples for the agentic workflows discussed in
[Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
by [Erik Schluntz](https://github.com/eschluntz) and [Barry Zhang](https://github.com/ItsBarryZ)
of Anthropic, inspired, ported and adapted from the
[code samples](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)
by the authors using [Pydantic AI](https://ai.pydantic.dev/).

## Orchestrator - Workers

In [ ]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
import json
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']
Using AI model: openai:gpt-4o


### Workflow: Orchestrator - Workers

An orchestrator is an agent that determines what work needs to be done
and divides it so that it can be delegated to multiple workers. Once
the work plan is ready, we run each task by a separate worker in parallel
and eventually collect the results from all workers.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> The schemas defined by Pydantic are translated into JSON schema, which is passed on
> to the model API for defining how the result should be structured. Each API handles
> this a bit differentls, but the details are abstracted by the library. In general,
> some complexity and nesting of types is usually OK. In this case, we define a `Task`
> type, and return a list of tasks as part of `OrchestratorResponse`.
>
> Agents in Pydantic AI have some fixed definitions, like their `system_prompt`, and
> can be used to execute multiple runs, each with its own prompt. Here we define a
> worker agent instance, which can execute each task definition.

In [3]:
class Task(BaseModel):
    type: str = Field(..., description=(
        'The type of task. '
        'For example: "formal", "conversational", "hybrid", ...'))
    description: str = Field(
        ...,
        description='Clear description for executing this task.'
    )


class OrchestratorResponse(BaseModel):
    analysis: str = Field(..., description=(
        'Explain your understanding of the task and which variations '
        'would be valuable. Focus on how each approach serves '
        'different aspects of the task.'
    ))
    tasks: List[Task] = Field(..., description="List of tasks")


async def orchestrate(task: str) -> Dict:
    """Process task by breaking it down and running subtasks in parallel."""

    orchestrator_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'Analyze this task and break it down into ',
            '2-3 distinct approaches.',
        ),
        result_type=OrchestratorResponse,
    )
    orchestrator_response = await orchestrator_agent.run(task)
    
    analysis = orchestrator_response.data.analysis
    tasks = orchestrator_response.data.tasks
    
    show('', title='Orchestrator Output')
    show(analysis, title='Analysis')
    show([task.model_dump() for task in tasks], title='Tasks')
    
    # Process all the tasks in parallel and collect results
    worker_agent = Agent(
        AI_MODEL,
        system_prompt='Generate content based on the task specification.',
    )
    worker_responses = await asyncio.gather(*[
        worker_agent.run(json.dumps(
            {'original_task': task} | task_info.model_dump()
        ))
        for task_info in tasks
    ])

    for task, response in zip(tasks, worker_responses):
        show(response.data, title=f"Worker Result ({task.type})")

In [4]:
await orchestrate(
    'Write a product description for a new eco-friendly water bottle.'
)


Orchestrator Output
-------------------


Analysis
--------

Creating a product description for a new eco-friendly water bottle involves focusing on its unique selling points, benefits to the consumer, and environmental impact. Different tones and styles can appeal to different market segments or purposes. A formal approach may be suitable for corporate presentations or websites targeting environmentally-conscious professionals. Conversational tones can engage a broader audience through storytelling and relatable language. Meanwhile, a hybrid approach can blend the professionalism of formal writing with the accessibility of a conversational style.


Tasks
-----

[{'description': 'Create a detailed and polished product description '
                 'emphasizing the eco-friendly aspects, material quality, and '
                 'health benefits of the new water bottle. This should cater '
                 'to an audience looking for sustainable and high-quality '
                 'prod